In [42]:
from scipy.spatial.distance import pdist, squareform
from scipy.stats import pearsonr
from numpy import arctanh
import numpy as np
import pandas as pd

In [43]:
behav = pd.read_csv('/Users/owenfriend/Documents/moshigo_local/submission/behavior.csv')
pca = pd.read_csv('/Users/owenfriend/Documents/moshigo_local/submission/pca_trajectories_hip.csv')

In [46]:
data = pd.DataFrame(columns = ['subject', 'age_group', 'age_num', 'run', 'z', 'navigation_accuracy', 'distance'])
for subject in pca.Subject.unique():
    for run in [1, 2, 3, 4, 5, 6]:
        sub_ref = pca[(pca['Subject'] == subject) & (pca['Run'] == run)]
        behav_ref = behav[(behav['subject'] == subject) & (behav['run'] == run)]
        age = sub_ref['AgeGroup'].values[0]
        if (len(sub_ref) > 0) & (len(behav_ref) > 0):    
            age_num = behav_ref['age'].values[0]
            run_coords_behav = []
            run_coords_pca = []
            for condition in [1, 2, 3, 4]:
                loc_ref_behav = behav_ref[behav_ref['condition'] == condition]

                ax = loc_ref_behav.goalx.values[0]
                ay = loc_ref_behav.goaly.values[0]
                cond_coords = [ax, ay]
                run_coords_behav.append(cond_coords)

                
                pca_ref = sub_ref[sub_ref['Item'] == condition]
                
                ax2 = pca_ref.PC1.values[0]
                ay2 = pca_ref.PC2.values[0]
                pca_coords = [ax2, ay2]
                run_coords_pca.append(pca_coords)

            A = np.array(run_coords_behav)
            B = np.array(run_coords_pca)
            dist_norm = squareform(pdist(A))
            dist_pca = squareform(pdist(B))
            idxs = np.triu_indices(4, k=1)
            r, p = pearsonr(dist_norm[idxs], dist_pca[idxs])
            z = arctanh(r)

            chance = np.mean(behav_ref['navigation_accuracy'])
            dist = np.mean(behav_ref['distance'])

            data.loc[len(data)] = [subject, age, age_num, run, z, chance, dist]
                

        

In [54]:
#data.to_csv('/Users/owenfriend/Documents/moshigo_local/submission/task_neural_space_corr.csv')